In [1]:
import pandas as pd
import numpy as np
from typing import *
import re
from three_answer import Answer
from functools import reduce
import operator
import fpgrowth_py
from functools import singledispatch

In [2]:
content: str = ""
read_content: str = ""
with open("output/result1_10k.txt", "r") as file:
    content=[ctnt.replace("*", "").strip() for ctnt in file.readlines()]

# with open("output/three_output.txt", "r") as file:
#     read_content=file.read()

INTERTIAL_vocabulary = ["sure", "I'd be happy", "happy", "please", "help", "given sentence", \
                        "I", "let", "based", "here are", "understood", "predictions", "mentioned", \
                            "sentence"]
escape_vocabulary=[re.escape(word) for word in INTERTIAL_vocabulary]
pattern = rf"\b({'|'.join(escape_vocabulary)})\b"

In [3]:
content[1]

'1. Project: Geological CO2 sequestration'

In [4]:
def list_split(cent: List[str] = content):
    index_int = 0
    for idx, ele in enumerate(cent):
        if re.search(r"^\d+$|\d$", ele):
            yield content[index_int: idx+1]
            index_int = idx+1
    return content[index_int: ]

In [5]:
res = list(list_split(content))

In [6]:
def digital_distinction(target: str) -> List[str]:
    """
    target here is supposed to be the last string of each response
    """
    match = re.search(pattern=r"\d+$", string=target)
    if not match: return [target] # for three_output.txt
    pos, dig= match.span(), match.group()
    return [target[:pos[0]], dig]

In [7]:
# res = [[*digital_distinction(sub_sec.pop(-1)), *sub_sec] for sub_sec in res]
for idx, sub_sec in enumerate(res):
    res[idx] = [sub_val \
                for val in map(digital_distinction, sub_sec) \
                    for sub_val in val]
# res = [[reduce(operator.concat, digital_distinction(val)) \
#               for val in sub_sec]\
#                      for sub_sec in res ]
# a bit problem, pop indeed prevent existence of extra the same value 
# but it makes the order of string into chaos

In [8]:
def interial_check(tar: Union[str, int])-> Union[str, Union[bool, int]]:
    if not isinstance(tar, str): return (None, False)
    digital = r"\d+$"
    numbers = re.search(pattern=digital, string=tar) # return the ditigal words hidden in the last senntece
    return re.search(pattern=pattern, string=tar, flags=re.IGNORECASE), int(numbers.group()) if numbers else False

In [9]:
def prefix_clean(string):
    if not isinstance(string, str): return string
    pattern = r"\* |\d+\.\s|\`"
    matches = re.sub(pattern=pattern, repl='', string=string, flags=re.MULTILINE)
    return matches

In [10]:
"""
structure of interial_check cannot be changed, better return a stable or single value 
"""
test_res = [[prefix_clean(sub_nele) \
             for sub_nele in nele \
                if not (not sub_nele or interial_check(sub_nele)[0])] \
                    for nele in res]

In [11]:
len(res)==len(test_res)
# "im gonna extract the word from result2_50k.txt according to a fixed pattern"

True

In [12]:
def int_geter(tar: List[Union[str, int]]):
    num = map(int, filter(lambda a: a.isdigit(), tar))
    num = next(iter(num))
    print(num, tar)
    tar.pop(tar.index(str(num)))
    return num, tar

In [13]:
# hb_head = r"^(stakeholder|risk|project)\b"
# re.search(hb_head, test_res[3740][2], re.IGNORECASE).span()[1]
# test_res[3740][2][12]

test_res[11]

['Project: Estimating the prevalence and odds of fleet driver errors and potentially distracting behaviors just prior to rear-end versus angle crashes.',
 'Risk: Inadequate surveillance, potentially distracting driver behaviors, spending too much time looking at or manipulating a cell phone while driving.',
 'Stakeholder: Fleet service drivers, as they are the ones who may benefit from',
 '10']

In [14]:
hb_head = r"^(stakeholder|risk|project)\b"
def horizaontal_detect(obj: List[str]):
    des = re.search(hb_head, obj, re.IGNORECASE)
    if des == None or obj[des.span()[1]+1] != "-": return False
    des = des.span()
    if obj[des[1]+1] == "-": 
        title, val = pattern_horizontal_bar(obj, des[1])
        return title+":"+val

def pattern_horizontal_bar(obj: str, pos: int):
    return obj[:pos], obj[pos+2: ]

In [15]:
template: List[str] = ["stakeholder:", "risk:", "project:", "stakeholders:", ": stakeholder", "category:"] #, ": risk", ": project"] #  
head_pattern = r"^(?:{})".format("|".join(template))
# https://stackoverflow.com/questions/62700774/singledispatchmethod-with-typing-types

class Answer_extraction:

    def __init__(self):
        pass

@singledispatch
def honer_solution(marker, obj: list):
    print("the string format is beyond the scope of the function")

@honer_solution.register
def _(marker: int, obj: list):
    """
    problem 1: index may not be able always keep the number in the last line
    problem 2: there will be extra explaination sentnece, find them out
    """
    global head_pattern
    three_ele, index,  = obj[:-1], obj[-1],
    extra: dict[str: list[str]] = {}

    for idx, _ in enumerate(three_ele):
        category = None
        while idx<len(three_ele) and not re.search(head_pattern, three_ele[idx], re.IGNORECASE):
            category_tuple = re.search(head_pattern, three_ele[idx-1], re.IGNORECASE)
            if not category: 
                start, end = category_tuple.span()
                if three_ele[idx-1][:3] == "Cat": category = three_ele[idx-1][end+1: ]
                else: category = three_ele[idx-1][start:end-1]
            if category not in extra.keys(): extra[category] = [three_ele.pop(idx)]
            else: extra[category].append(three_ele.pop(idx))
            
    if len(extra):
        return extra, [], index

    three_ele = [ab_ele.split(":") for ab_ele in three_ele]
    return {real_time[0]: [*(real_time[1].split(",", -1) if real_time[1] else [])] for real_time in three_ele}, extra, index
    

In [16]:
qualified_col_name = ["risk", "stakeholder", "project"]
def name_correct(val: str):
    for name in qualified_col_name:
        if name in val.lower(): return name
    return val

@honer_solution.register
def _(marker: str, obj: list):
    """
    only solve answer that in one line. e.g
    Sure! Based on the given sentence, here are the three categories I would assign:
    * Risk: droughts
    * Stakeholder: forest ecosystem, climate change
    * Project: simulation of changes in live carbon stocks in the Amazon Basin
    4
    """
    global head_pattern
    three_ele, index,  = obj[:-1], obj[-1],
    extra: dict[str: list[str]] = {}
    try:
        three_ele = [ab_ele.split(":") for ab_ele in three_ele]
        for real_time in three_ele:
            if not real_time[1]: real_time[1] = []
            else: real_time[1] = [re.sub(r'^\W+|\W+$', '', val) for val in real_time[1].split(",", -1)]
        return {name_correct(real_time[0]): real_time[1] for real_time in three_ele}, extra, index
    except Exception as e:
        return False, e, index

In [17]:
test_res[0]

['Project: Geological CO2 sequestration',
 'Stakeholders:',
 'Risk: Mitigation of anthropogenic carbon dioxide emissions, injection capacity and safety of the storage system, fluid flow in porous media, etc.',
 '0']

In [18]:
test_num = 0
print(honer_solution("0", test_res[test_num]))

({'project': ['Geological CO2 sequestration'], 'stakeholder': [], 'risk': ['Mitigation of anthropogenic carbon dioxide emissions', 'injection capacity and safety of the storage system', 'fluid flow in porous media', 'etc']}, {}, '0')


In [19]:
abs_data = []
abnormal = []
for nuidx, val in enumerate(test_res):
    val, extra, ind = honer_solution("0", val)
    if not val:
        abnormal.append(nuidx)
    else:
        abs_data.append((val, extra, ind))
print(len(abnormal))
# possible worked for index 3740
pattern = fr"\b({'|'.join(template)})\s*[:-]"

928


In [20]:
def abnormal_record(obj: List[int] = abnormal):
    global res
    return [res[idx][-1] for idx in obj]

# print(test_res[int(abnormal[5])], "\n", res[int(abnormal[5])])
res[int(abnormal[5])][-1]

'70'

In [21]:
abnormal_path = rf"output/abnormal_{len(abnormal)}_record.txt"
with open(abnormal_path, "w") as file:
    file.write("\n".join(abnormal_record()))

In [116]:
hder, std_col = True, ["project", "risk", "stakeholder", "idx"]
for idv, order_ele in enumerate(abs_data):
    normal, extra, index = order_ele
    if not bool(normal): continue
    normal["idx"] = [index]
    normal = {key: ['|'.join(value).strip()] if isinstance(value, list) else str(value) for key, value in normal.items()}
    fuck_off = pd.DataFrame(normal)
    if len(fuck_off) < 4:
        fuck_off = fuck_off.reindex(columns=std_col).fillna("None")
    try: 
        fuck_off.to_csv("Generated_Result/1k.csv", index=False, mode="a", header=hder)
        hder = False
    except Exception as e:
        print(idv)

In [123]:
abs_data[20]

({'project': ['Gravel road safety'],
  'risk': ['Inherent hazards present in gravel roads',
   'lack of research related to crash count prediction models for these roads',
   'and difficulty in applying existing methods due to unique conditions in Laramie County',
   'Wyoming'],
  'stakeholder': ['Wyoming Department of Transportation',
   'safety experts',
   'and residents who use gravel roads'],
  'idx': ['19']},
 {},
 '19')

In [127]:
result = pd.read_csv(r"./Generated_Result/1k.csv", sep=";", header=0).fillna("None")
result.head()

,project,risk,stakeholder,idx
0,Geological CO2 sequestration,Mitigation of anthropogenic carbon dioxide emi...,None,0.0
1,development and validation of a discrete event...,None,commercial vehicles passing through the area,1.0
2,Urban renaissance project in the area of Gazi,Pollution from the industrial use of Gazi|whic...,Residents of Gazi|environmental activists|loca...,2.0
3,simulation of changes in live carbon stocks in...,droughts,forest ecosystem|climate change,4.0
4,Ecological enhancement of coastal and marine i...,degraded or reduced habitat for native species,ecosystem and habitat value|this paper present...,5.0


In [139]:
## detect how many useless element in each of column and get its index
## useless element is words which inolves word "None"
for col in result:
    indices = [i for i, val in enumerate(result[col]) if "None" in str(val)]
    print(f"Column '{col}' has {len(indices)} useless element(s) compare with {result.shape[0]} /10_000 \n\
          {round(len(indices)/result.shape[0], 2)} at indices: {indices}")

Column 'project' has 763 useless element(s) compare with 9085 /10_000 
          0.08 at indices: [7, 9, 11, 12, 14, 15, 37, 40, 42, 67, 77, 89, 107, 114, 115, 116, 123, 159, 162, 178, 208, 225, 233, 238, 250, 254, 257, 293, 294, 337, 350, 357, 358, 360, 364, 411, 425, 482, 483, 491, 496, 505, 517, 521, 568, 580, 591, 596, 601, 603, 607, 618, 623, 638, 644, 671, 681, 684, 693, 711, 714, 729, 731, 754, 757, 769, 797, 814, 846, 847, 851, 855, 856, 861, 871, 874, 882, 899, 904, 907, 911, 917, 925, 944, 945, 947, 964, 966, 972, 978, 990, 991, 994, 1014, 1040, 1041, 1069, 1083, 1092, 1113, 1117, 1122, 1135, 1136, 1144, 1155, 1161, 1189, 1206, 1230, 1234, 1248, 1249, 1261, 1266, 1277, 1285, 1286, 1287, 1292, 1299, 1333, 1344, 1345, 1352, 1362, 1368, 1370, 1391, 1397, 1401, 1410, 1422, 1426, 1434, 1436, 1443, 1453, 1470, 1471, 1485, 1535, 1540, 1546, 1550, 1560, 1572, 1576, 1582, 1585, 1592, 1609, 1610, 1638, 1643, 1653, 1656, 1659, 1662, 1709, 1710, 1712, 1727, 1735, 1740, 1744, 1754, 1760, 

In [133]:
result.shape[0]
# {real_time[0]: [*(real_time[1].split(", ", -1) if real_time[1] else [])] for real_time in three_ele}, index

9085